In [ ]:
import sys
sys.path.append("../DeepHypergraph/") 

In [ ]:
import time
from copy import deepcopy

import torch
import torch.optim as optim
import torch.nn.functional as F

from dhg import Hypergraph
from dhg.data import DBLP8k
from dhg.models import HGNNPLinkPred
from dhg.random import set_seed
from dhg.metrics import LinkPredictionEvaluator as Evaluator

In [ ]:
def train(net, X, hypergraph, negative_hypergraph, optimizer, epoch):
    net.train()

    st = time.time()
    optimizer.zero_grad()
    pos_score = net(X, hypergraph)
    neg_score = net(X, negative_hypergraph)

    global device
    scores = torch.cat([pos_score, neg_score]).squeeze()
    labels = torch.cat(
        [torch.ones(pos_score.shape[0]), torch.zeros(neg_score.shape[0])]
    ).to(device)

    loss = F.binary_cross_entropy_with_logits(scores, labels)
    loss.backward()
    optimizer.step()
    print(f"Epoch: {epoch}, Time: {time.time()-st:.5f}s, Loss: {loss.item():.5f}")
    return loss.item()

In [ ]:
@torch.no_grad()
def infer(net, X, hypergraph, negative_hypergraph, test=False):
    net.eval()
    pos_score = net(X, hypergraph)
    neg_score = net(X, negative_hypergraph)

    scores = torch.cat([pos_score, neg_score]).squeeze()
    labels = torch.cat(
        [torch.ones(pos_score.shape[0]), torch.zeros(neg_score.shape[0])]
    ).to(device)

    global evaluator
    if not test:
        res = evaluator.validate(labels, scores)
    else:
        res = evaluator.test(labels, scores)
    return res

In [ ]:
import csv
from pathlib import Path

def load_data(file_path: Path):
    hyperedge_list = []
    neg_hyperedge_list = []
    with open(file_path / "hyperedges.csv", "r") as file:
        reader = csv.reader(file)
        for row in reader:
            # 读取每个超边的顶点列表，并将它们添加到 hyperedge_list 中
            hyperedge_list.append(row)
    
    hyperedge_list = [[int(v) for v in edge] for edge in hyperedge_list]

    with open(file_path / "minimal_unschedulable_combinations.csv", "r") as file:
        reader = csv.reader(file)
        for row in reader:
            neg_hyperedge_list.append(row) 

    neg_hyperedge_list = [[int(v) for v in edge] for edge in neg_hyperedge_list]

    with open(file_path / "task_quadruples.csv", 'r') as csvfile:
        reader = csv.reader(csvfile)
        data = [list(map(float, row)) for row in reader]

    # 将数据转换为 Tensor
    features = torch.tensor(data)

    data = {"hyperedge_list": hyperedge_list, "num_edges" : len(hyperedge_list)}
    neg_data = {"hyperedge_list": neg_hyperedge_list, "num_edges" : len(neg_hyperedge_list)}

    return {"pos":data, "neg": neg_data, "vertices_feature" : features, "num_vertices" : features.shape[0]}

In [ ]:
set_seed(2021)
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
evaluator = Evaluator(["accuracy", "auc", "f1_score"])
train_data =    load_data(Path("../EDF/data/data_s3000_p3_t1000_hs7_e10000/"))
validate_data = load_data(Path("../EDF/data/data_s3001_p3_t1000_hs7_e10000/"))
test_data =     load_data(Path("../EDF/data/data_s3002_p3_t1000_hs7_e10000/"))

X = train_data["vertices_feature"]
HG = Hypergraph(train_data["num_vertices"], train_data["pos"]["hyperedge_list"])
neg_HG = Hypergraph(train_data["num_vertices"], train_data["neg"]["hyperedge_list"])

validate_X = validate_data["vertices_feature"]
validate_HG = Hypergraph(validate_data["num_vertices"], validate_data["pos"]["hyperedge_list"])
validate_neg_HG = Hypergraph(validate_data["num_vertices"], validate_data["neg"]["hyperedge_list"])

test_X = test_data["vertices_feature"]
test_HG = Hypergraph(test_data["num_vertices"], test_data["pos"]["hyperedge_list"])
test_neg_HG = Hypergraph(test_data["num_vertices"], test_data["neg"]["hyperedge_list"])

In [ ]:
import numpy as np

def calculate_sparsity(matrix):
    nonzero_elements = np.count_nonzero(matrix)
    total_elements = matrix.size

    nonzero_ratio = nonzero_elements / total_elements
    zero_ratio = 1 - nonzero_ratio

    print(f"非零元素比例：{nonzero_ratio:.2%}")
    print(f"零元素比例：{zero_ratio:.2%}")

calculate_sparsity(HG.H.to_dense().cpu().numpy())

In [ ]:
degree_values = HG.deg_v
# degree_values = validate_HG.deg_v
# degree_values = test_HG.deg_v
print(degree_values)
print(degree_values[489])

In [ ]:
import matplotlib.pyplot as plt
# 绘制节点度数的柱状图
plt.bar(range(len(degree_values)), degree_values)
plt.xticks(range(len(degree_values)), range(len(degree_values)))
plt.xlabel('Node Index')
plt.ylabel('Degree')
plt.title('Degree Distribution')
plt.show()